<a href="https://colab.research.google.com/github/LucasRBerenger/brasileirao-standings-metrics/blob/main/brasileirao_standings_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv("matches-2003-2025.txt", sep=";", encoding="latin-1") # especifying encoding because csv has latin caracters
df.to_csv("brasileirao_matches.csv", index = False) # creating df_matches (1)

print(df.columns.tolist())
df.head()

['#DATE', 'HOUR', 'HOME-TEAM', 'SCORE', 'AWAY-TEAM']


,#DATE,HOUR,HOME-TEAM,SCORE,AWAY-TEAM
0,29/03/2003,18:00,Guarani/SP,4x2,Vasco/RJ
1,29/03/2003,18:00,Atlético/PR,2x0,Grêmio/RS
2,30/03/2003,18:00,Goiás/GO,2x2,Paysandu/PA
3,30/03/2003,18:00,Corinthians/SP,0x3,Atlético/MG
4,30/03/2003,16:00,Flamengo/RJ,1x1,Coritiba/PR


In [2]:
df = pd.read_csv("brasileirao_matches.csv", sep = ",") # data processing
df.head()

#  data cleaning, to remove strange characters, prefix/suffix space and to turn columns upper case
df.columns = df.columns.str.replace("#", "").str.strip().str.upper() # columns are already upper case, but is a good pratice to garantee

# logic to transform df
df["DATE"] = pd.to_datetime(df["DATE"], dayfirst = True)
df["YEAR"] = df["DATE"].dt.year

df[["HOME_GOALS", "AWAY_GOALS"]] = df["SCORE"].str.split("x", expand = True).astype(int)

merged_df = [] # empty df with teams performance

# df_matches will be turned into a more valiable df for ranking
for i, row in df.iterrows():

  # rows for merged_df
  year = row["YEAR"]
  home = row["HOME-TEAM"]
  away = row["AWAY-TEAM"]
  home_goals = row["HOME_GOALS"]
  away_goals = row["AWAY_GOALS"]

  home_team = { "Team": home, "Pts": 0, "MP": 0, "W": 0, "D": 0, "L": 0, "GF": home_goals, "GA": away_goals, "GD": 0, "Year": year} # df1
  away_team = { "Team": away, "Pts": 0, "MP": 0, "W": 0, "D": 0, "L": 0, "GF": away_goals, "GA": home_goals, "GD": 0, "Year": year} # df2

  if home_goals > away_goals:
    home_team["Pts"] = 3
    away_team["Pts"] = 0
    home_team["W"] = 1
    away_team["L"] = 1
  elif away_goals > home_goals:
    home_team["Pts"] = 0
    away_team["Pts"] = 3
    home_team["L"] = 1
    away_team["W"] = 1
  else:
    home_team["Pts"] = 1
    away_team["Pts"] = 1
    home_team["D"] = 1
    away_team["D"] = 1

  home_team["MP"] = 1
  away_team["MP"] = 1

  merged_df.append(home_team)
  merged_df.append(away_team)

merged_df = pd.DataFrame(merged_df) # intermediary df finally merged with all processed data (2)

# print(merged_df)

ranking = merged_df.groupby(["Year", "Team"]).sum().reset_index() # grouping rows by TEAM and Year, df (3)

ranking["GD"] = ranking["GF"] - ranking["GA"]

ranking = ranking.sort_values(by = ["Year", "Pts", "W", "GD", "GF"], ascending = [True, False, False, False, False]) # (CRESCENTE true or false)

ranking['Rank'] = ranking.groupby('Year').cumcount() + 1 # ranking 1,2,3... per year

# print(ranking)
# ranking.to_csv("brasileirao_rankings.csv", index = False)

columns = list(ranking.columns)
columns.remove("Rank")
columns.remove("Year")
ranking = ranking[["Year"] + ["Rank"] + columns]

final_ranking = ranking[ranking["Year"] >= 2006] # selecting just brazilian league modern era (2006 - 2025), final df (4)
final_ranking.to_csv("brazilian_league.csv", index=False)

